In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
import gc
import os
from math import floor, ceil
import tensorflow.keras as keras
import tensorflow_hub as hub
from scipy.stats import spearmanr

np.set_printoptions(suppress=True)
disable_eager_execution()

In [ ]:
embed = hub.Module('module_useT/')

In [8]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv('test.csv')
print('train shape: {}'.format(df_train.shape))
print('test shape: {}'.format(df_test.shape))

train shape: (6079, 41)
test shape: (476, 11)


In [11]:
output_categories = list(df_train.columns[11:])
input_categories = list(df_train.columns[[1, 2, 5, 9, 10]])
print('\nOutput Categories:\n\t', output_categories)
print('\nInput Categories:\n\t', input_categories)


Output Categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

Input Categories:
	 ['question_title', 'question_body', 'answer', 'category', 'host']


In [4]:
train_data.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [20]:
def convert_to_use_vectors(df, embed):
    t = np.empty((len(df), 512))
    q = np.empty((len(df), 512))
    a = np.empty((len(df), 512))
    for i, instance in tqdm(df.iterrows()):
        t_constant = tf.constant([str(instance.question_title)])
        q_constant = tf.constant([str(instance.question_body)])
        a_constant = tf.constant([str(instance.answer)])
#         t[i, :] = embed.signatures['default'](t_constant)
        print(embed.signatures['default'](t_constant).keys())
#         q[i, :] = embed.signatures['default'](q_constant).numpy().flatten()
#         a[i, :] = embed.signatures['default'](a_constant).numpy().flatten()
    return t, q, a

embed = hub.load('module_useT/')

train_t, train_q, train_a = convert_to_use_vectors(df_train, embed)
test_t, test_q, test_a = convert_to_use_vectors(df_test, embed)

dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys([

dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys([

dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys([

dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys([

dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys([

...

dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys([

dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys([

dict_keys(['default'])



dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys([

dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys(['default'])
dict_keys([